In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

e:\VS Code\GPTSummary\.env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
model_name_or_path = "sberbank-ai/rugpt2large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

In [4]:
def add_special_tokens():
    special_tokens = {'pad_token': '|<pad>|', 'sep_token': '|<sep>|'}
    num_add_tokens = tokenizer.add_special_tokens(special_tokens)
    return tokenizer

In [5]:
import torch
import pandas as pd
from torch.utils.data import Dataset

class GPT22048Dataset(Dataset):
    def __init__(self, path, mode='train'):
        self.tokenizer: GPT2Tokenizer = add_special_tokens()
        self.mode = mode
        self.df = pd.read_json(path, lines=True).drop(columns=['date', 'url', 'title'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.tokenizer.encode(self.tokenizer.pad_token) * 2048
        content = self.df['text'][idx] + self.tokenizer.encode(self.tokenizer.sep_token) + self.df['summary'][idx]
        text[:len(content)] = content
        text = torch.tensor(text)
        sample = {'text': text, 'sum_idx': len(self.df['text'])}

        return sample

In [16]:
import random

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [7]:
import os
import datetime

def evaluate(model, eval_dataset, ignore_index, global_step=None):
    """ Returns perplexity score on validation dataset.
        Args:
            args: dict that contains all the necessary information passed by user while training
            model: finetuned gpt/gpt2 model
            eval_dataset: GPT21024Dataset object for validation data
            global_step: no. of times gradients have backpropagated
            ignore_index: token not considered in loss calculation
    """
    if not os.path.exists('output_dir'):
        os.mkdir('output_dir')
    eval_output_dir = 'output_dir'

    results = {}
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=32)
    loss_fct = CrossEntropyLoss(ignore_index=ignore_index) #ignores padding token for loss calculation

    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = batch['article'].to(device), batch['article'].to(device)
        
        with torch.no_grad():
            logits = model(inputs)[0]
            idx = batch['sum_idx'].item() # index of separator token
            # only consider loss on reference summary just like seq2seq models
            shift_logits = logits[..., batch['sum_idx']:-1, :].contiguous()
            shift_labels = labels[..., batch['sum_idx']+1:].contiguous()
            lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {
        "perplexity": perplexity
    }
    print("perplexity:", perplexity.item())

    if global_step:
        output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
        with open(output_eval_file, "a") as f:
            for key in sorted(result.keys()):
                f.write('\n\n')
                f.write("time = %s, %s = %s, step = %s\n" % (datetime.now().strftime("%d/%m/%Y %H:%M:%S"), key, str(result[key]), str(global_step)))
    return result           

In [22]:
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from torch.optim import AdamW


def train(model, tokenizer, train_dataset, valid_dataset, ignore_index):
    """ Trains GPT2 model and logs necessary details.
        Args:
            args: dict that contains all the necessary information passed by user while training
            model: finetuned gpt/gpt2 model
            tokenizer: GPT/GPT2 tokenizer
            train_dataset: GPT21024Dataset object for training data
            ignore_index: token not considered in loss calculation
    """
    train_sampler = RandomSampler(train_dataset)
    train_dl = DataLoader(train_dataset, sampler=train_sampler,
                          batch_size=32, num_workers=4)
    # ignores padding token for loss calculation
    loss_fct = CrossEntropyLoss(ignore_index=ignore_index)
    optimizer = AdamW(model.parameters(), lr=0.003)
    scheduler = get_linear_schedule_with_warmup(optimizer, 100, 80000)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    set_seed(15451)
    for _ in tqdm(range(50), desc="Epoch"):
        epoch_iterator = tqdm(train_dl, desc="Training")
        for step, batch in enumerate(epoch_iterator):
            inputs, labels = batch['article'].to(device), batch['article'].to(device)
            model.train()
            logits = model(inputs)[0]
            # only consider loss on reference summary just like seq2seq models
            shift_logits = logits[..., batch['sum_idx']:-1, :].contiguous()
            shift_labels = labels[..., batch['sum_idx']+1:].contiguous()
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss = loss/32
            loss.backward()
            torch.nn.utils.clip_grad_norm_(
                model.parameters(), 1)
            tr_loss += loss.item()
            if (step + 1) % 32 == 0:
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1
                logging_loss = tr_loss
                print("loss:", loss.item(), end='\n\n')
                if (step + 1)/32 == 1.0:
                    print('After 1st update: ', end='\n\n')
            
            if (step + 1) % (10*32) == 0:
                results = evaluate(model, valid_dataset,
                                   ignore_index, global_step)
                print('After', global_step+1, 'updates: ', end='\n\n')


In [9]:
train_data = GPT22048Dataset('gazeta_train.jsonl', mode='train')
valid_data = GPT22048Dataset('gazeta_test.jsonl', mode='valid')

In [10]:
tokenizer = add_special_tokens()
ignore_idx = tokenizer.pad_token_id

In [11]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 1280)

In [12]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [23]:
import time

start = time.time()
train(model, tokenizer, train_data, valid_data, ignore_idx)
print('total time: ', (time.time()-start)/60, " minutes", end='\n\n')

e:\VS Code\GPTSummary\.env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

